## library import

In [1]:
# Data manipulation
import pandas as pd
import numpy as np
import re

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30
pd.options.display.max_colwidth = None

## Constants

In [2]:
# list of new discovered features from address and benefits
col_add_address = [
    'Company location (Area)',
    'Company location (suburb)',
    'Salary($)(Benefit)'
]

In [3]:
# list of new discovered features from job class
col_add_jobClass = [
    'job_classification',
    'job_subclassification'
]

In [4]:
# list of new discovered features from salary
col_add_salary = [
    'salary_range',
    'salary_range_filled',
    'salary_min',
    'salary_max'
]

In [5]:
# assgin subclass for some occupation especially for "unknown" and "others"
occupation_subclass_dict = {
    'Category Manager': 'Office Management',
    'Temporary Office Roles': 'Contracts Administration',
    'Executive Assistant': 'PA, EA & Secretarial',
    'Administrator - Temps': 'Contracts Administration'
}

## Help functions

In [6]:
# rename dataset features with readable names
def renameCol(df):
    return df.rename({
                '字段1': 'column1', 
                '字段1_link': 'column1_link',
                '字段2': 'column2', 
                '字段3': 'column3',
                '字段4': 'column4', 
                '字段5': 'column5'
            }, axis=1, inplace=True)

#### Address help function

In [7]:
# extracting area from full address
def address_area(address):
    lst_address = [x.strip() for x in address.split(":")][1]
    lst_area = re.findall('[A-Z][^A-Z]*', lst_address)
    area = ""
    for i in range(len(lst_area)//2):
        area += lst_area[i]
    return area

In [8]:
# extracting suburb from full address
def address_suburb(address):
    lst_address = [x.strip() for x in address.split(":")]
    if len(lst_address) == 3:
        lst_suburb = re.findall('[A-Z][^A-Z]*', lst_address[2])
        suburb = ""
        for i in range(len(lst_suburb)//2):
            suburb += lst_suburb[i]
        return suburb

#### Benefit help function

In [9]:
# extract benefits including salary, health benefits, bonus, holidays and so on.
def address_benefit(address):
    benefit = address.split(',', 1)
    if len(benefit)>1:
        return benefit[1]

#### Job posted help function

In [10]:
def job_posted_days(row):
    try:
        if re.search(r'(\d*)(\w)', re.search(r'(\d+)(\w)', row).group()).group(2) == 'd':
            return re.search(r'(\d*)(\w)', re.search(r'(\d+)(\w)', row).group()).group(1)
        else:
            return 0
    except:
      return None

#### Job class help function

In [11]:
# extracting job class
def return_classification(row):
    lst_row = row.split(':')
    classfication = ""
    if len(lst_row)==1:
        return None
    else:
        lst_class = re.findall('[A-Z][^A-Z]*', lst_row[1].split('subClassfication')[0])
        for i in range(len(lst_class)//2):
            classfication += lst_class[i]
        return classfication

In [12]:
# extracting job subclass
def return_subclass(row):
    lst_row = row.split(':')
    subclass = ""
    if len(lst_row)==1:
        return None
    else:
        lst_subclass = re.findall('[A-Z][^A-Z]*', lst_row[2])
        for i in range(len(lst_subclass)//2):
            subclass += lst_subclass[i]
        return subclass

#### Salary help function

In [13]:
# filled missing values of minimum and maximum salary
### Normally, they should be filled by reference tables
def salaryMinMax(row):
    try:
        s_min = row[0]
        s_max = row[1]
        return [int(s_min), int(s_max)]
    except:
        s_min = min(np.random.randint(45000, 75000, size=2))
        s_max = max(np.random.randint(45000, 75000, size=2))
        return [s_min, s_max]

In [14]:
# return a tuple containing minimum and maximum salary from salary range which is 
# orginally from benefit feature
def salary_extract(s):
    temp1 = re.findall('.*\d+[.]*[,]*\d*k*K*', s)
    if len(temp1) != 0:
        temp2 = re.findall('\d+\s*[.]*[,]*\d*k*K*', temp1[0])
        if len(temp2) == 1 and len(temp2[0]) > 1:
            if ',' in temp2[0]:
                return ([float(temp2[0].replace(',', '')), float(temp2[0].replace(',', ''))])
            elif 'k' in temp2[0]:
                return ([float(temp2[0].replace('k','000')), float(temp2[0].replace('k','000'))])
            else:
                if float(temp2[0]) > 300:
                    return ([float(temp2[0]), float(temp2[0])])
                else:
                    return ([float(temp2[0])*8*200, float(temp2[0])*8*200])
        elif len(temp2) > 1:
            if ',' in temp2[0] or ',' in temp2[1]:
                if ',' in temp2[0] and ',' in temp2[0]:
                    return ([float(temp2[0].replace(',', '')), float(temp2[1].replace(',', ''))])
                elif ',' in temp2[0] and ',' not in temp2[1]:
                    return ([float(temp2[0].replace(',000', ''))*1000, float(temp2[1])]*1000)
                elif ',' in temp2[1] and ',' not in temp2[0]:
                    return ([float(temp2[0])*1000, float(temp2[1].replace(',000', ''))*1000])
            elif 'k' in temp2[0] or 'k' in temp2[1]:
                if 'k' in temp2[0] and 'k' in temp2[0]:
                    return ([float(temp2[0].replace('k', '000')), float(temp2[1].replace('k', '000'))])
                elif 'k' in temp2[0] and 'k' not in temp2[1]:
                    return ([float(temp2[0].replace('k', '000')), float(temp2[1])*1000])
                elif 'k' in temp2[1] and 'k' not in temp2[0]:
                    return ([float(temp2[0])*1000, float(temp2[1].replace('k', '000'))])
            elif 'K' in temp2[0] or 'K' in temp2[1]:
                if 'K' in temp2[0] and 'K' in temp2[0]:
                    return ([float(temp2[0].replace('K', '000')), float(temp2[1].replace('K', '000'))])
                elif 'K' in temp2[0] and 'K' not in temp2[1]:
                    return ([float(temp2[0].replace('K', '000')), float(temp2[1])*1000])
                elif 'K' in temp2[1] and 'K' not in temp2[0]:
                    return ([float(temp2[0])*1000, float(temp2[1].replace('K', '000'))])
            elif temp2[1] == '8':
                return ([float(temp2[0])*8*200, float(temp2[0])*8*200])
            else:
                if ' ' in temp2[0] or ' ' in temp2[1]:
                    if ' ' in temp2[0] and ' ' in temp2[1]:
                        if len(temp2[0])>4 and len(temp2[1])>4:
                            return ([float(temp2[0].replace(' ', '')), float(temp2[1].replace(' ', ''))])
                        else:
                            return ([float(temp2[0].replace(' ', ''))*8*200, float(temp2[1].replace(' ', ''))*8*200])
                    elif ' ' in temp2[0] and ' ' not in temp2[1]:
                        return ([float(temp2[0].replace(' ', ''))*8*200, float(temp2[1])*8*200])
                    if ' ' not in temp2[0] and ' ' in temp2[1]:
                        return ([float(temp2[0])*8*200, float(temp2[1].replace(' ', ''))*8*200])
                else:
                    return ([float(temp2[0])*8*200, float(temp2[1])*8*200])

## Clean processing functions

### Clean link

In [15]:
# Job adverts have different promotion types such as standard (free) and promoted (premium)
def cleanCol1_link(df, col_edit, col_add):
    df[col_add] = df[col_edit].apply(lambda row: re.search(r'=(\w+)#', row).group(1))
    return df

### Clean address and benefits

In [16]:
def clean_address_benefit(df, col_edit, col_add_address):
    df[col_add_address[0]] = df[col_edit].apply(lambda row: address_area(row))
    df[col_add_address[1]] = df[col_edit].apply(lambda row: address_suburb(row))
    df[col_add_address[2]] = df[col_edit].apply(lambda row: address_benefit(row))
    return df

### Clean job posted

In [17]:
def clean_jobPosted(df, col_edit, col_add, func):
    df[col_add] = df[col_edit].apply(lambda row: func(row))
    return df

### Clean Job class

In [18]:
def clean_jobClass(df, col_edit, col_add_jobClass):
    df[col_add_jobClass[0]] = df[col_edit].apply(lambda row: return_classification(row))
    df[col_add_jobClass[1]] = df[col_edit].apply(lambda row: return_subclass(row))
    return df

### Clean salary

In [19]:
def clean_salary(df, col_edit, col_add_salary):
    df[col_add_salary[0]] = df[col_edit].apply(lambda row: salary_extract(str(row)))
    df[col_add_salary[1]] = df[col_add_salary[0]].apply(lambda row: salaryMinMax(row))
    df[[col_add_salary[2], col_add_salary[3]]] = df[col_add_salary[1]].apply(pd.Series)
    return df

### Clean redundancies

In [20]:
def clean_redundancies(df):
    df.rename({'column1': 'occupation', 'column2': 'Company Name'}, axis=1, inplace=True)
    df['Company Name'].fillna(value="Private Advertiser", inplace=True)
    df.drop([
        'salary_range', 
        'salary_range_filled', 
        'Salary($)(Benefit)',
        'column1_link', 
        'column3', 
        'column4', 
        'column5'], axis=1, inplace=True)
    df['Company location (suburb)'].fillna("unknown", inplace=True)
    df['Job_Posted(days ago)'].fillna("-1", inplace=True)
    df['job_classification'].fillna('unknown', inplace=True)
    df['job_subclassification'].fillna('unknown', inplace=True)
    # assgin subclass for some occupation especially for "unknown" and "others"
    for index, value in enumerate(df['occupation']):
        if df.iloc[index]['occupation'] in occupation_subclass_dict.keys():
            df.at[index, 'job_subclassification'] = occupation_subclass_dict[df.iloc[index]['occupation']]
    return df

### Main clean function

In [21]:
def main_clean(path):
    df = pd.read_excel(path)
    renameCol(df)
    df = cleanCol1_link(df, 'column1_link', 'job_advert_type')
    df = clean_address_benefit(df, 'column3', col_add_address)
    df = clean_jobPosted(df, 'column4', 'Job_Posted(days ago)', job_posted_days)
    df = clean_jobClass(df, 'column5', col_add_jobClass)
    df = clean_salary(df, 'Salary($)(Benefit)', col_add_salary)
    df = clean_redundancies(df)
    return df

In [22]:
df_admin = main_clean('NZ_Admin_JOBS.xlsx')
df_admin.head(10)

,occupation,Company Name,job_advert_type,Company location (Area),Company location (suburb),Job_Posted(days ago),job_classification,job_subclassification,salary_min,salary_max
0,Administrator,Private Advertiser,promoted,Bay of Plenty,Tauranga,-1,Administration & Office Support,Office Management,49461,63898
1,Receptionist,Avenues Orthodontics,promoted,Bay of Plenty,Tauranga,-1,Administration & Office Support,Receptionists,46375,73414
2,Prosecutions Support Officer,New Zealand Police,standard,Auckland,unknown,4,Administration & Office Support,Other,51855,74645
3,Early Childhood Centre Administrator,Kew Pacific Island Early Learning Centre,standard,Southland,Invercargill,0,Administration & Office Support,Administrative Assistants,64852,71353
4,Business Support Administrator,Private Advertiser,standout,Canterbury,Christchurch,4,Administration & Office Support,Client & Sales Administration,49499,61636
5,Support Officer,Ministry for Primary Industries,standout,Northland,Whangarei,0,Administration & Office Support,Administrative Assistants,52592,67235
6,Support Officer,"Ministry of Business, Innovation and Employment",standout,Wellington,Wellington Central,5,Administration & Office Support,Other,51565,69111
7,office administrator,Hepburn Electrical Ltd,standard,Bay of Plenty,Rotorua,0,Administration & Office Support,Administrative Assistants,47454,72022
8,Office Administrator,Webster Holland Ltd,standard,Bay of Plenty,Tauranga,0,Administration & Office Support,Administrative Assistants,60331,57408
9,Administration Officer,New Zealand Police,standard,Canterbury,unknown,3,Administration & Office Support,Other,53174,66084


In [23]:
df_admin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2708 entries, 0 to 2707
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   occupation                 2708 non-null   object
 1   Company Name               2708 non-null   object
 2   job_advert_type            2708 non-null   object
 3   Company location (Area)    2708 non-null   object
 4   Company location (suburb)  2708 non-null   object
 5   Job_Posted(days ago)       2708 non-null   object
 6   job_classification         2708 non-null   object
 7   job_subclassification      2708 non-null   object
 8   salary_min                 2708 non-null   int64 
 9   salary_max                 2708 non-null   int64 
dtypes: int64(2), object(8)
memory usage: 211.7+ KB


In [24]:
df_admin['job_subclassification'].value_counts().head(20)

Administrative Assistants                715
PA, EA & Secretarial                     580
Contracts Administration                 449
Client & Sales Administration            336
Receptionists                            196
Office Management                        182
unknown                                  110
Records Management & Document Control     81
Other                                     46
Data Entry & Word Processing              13
Name: job_subclassification, dtype: int64